In [1]:
import mlflow

In [2]:
import gdown

In [3]:
import os

In [16]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from pathlib import Path
from src.textclassifier.entity.config_entity import TrainingConfig
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout



In [4]:
%pwd

'/Users/ayushyajnik/Desktop/Python/textclassification/fakeNewsClassifier/research'

In [5]:
os.chdir('../')

In [ ]:
MLFLOW_TRACKING_URI=https://dagshub.com/ayajnik/fakeNewsClassifier.mlflow \
MLFLOW_TRACKING_USERNAME=ayajnik \
MLFLOW_TRACKING_PASSWORD=747fee523fd3d119cd9cdec23c57e86deb8cd7ce \
python script.py

In [7]:
import tensorflow as tf


/Users/ayushyajnik/anaconda3/lib/python3.11/site-packages/tensorflow/python/framework/dtypes.py:35: DeprecationWarning: ml_dtypes.float8_e4m3b11 is deprecated. Use ml_dtypes.float8_e4m3b11fnuz
  from tensorflow.tsl.python.lib.core import pywrap_ml_dtypes


In [8]:
model = tf.keras.models.load_model("artifacts/training/model.h5")


2023-12-19 15:39:00.421482: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-12-19 15:39:00.421515: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-12-19 15:39:00.421523: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-12-19 15:39:00.421607: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-19 15:39:00.421658: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [24]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    #params_batch_size: int
    params_is_epochs: int
    params_is_batch_size: int
    params_is_vocab: int
    params_is_sent_length: int
    params_is_sen_length: int

In [25]:
from src.textclassifier.constants import *
from src.textclassifier.utils.common import read_yaml, create_directories, save_json

In [26]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="",
            training_data="",
            mlflow_uri="",
            all_params=self.params,
            #params_batch_size=self.params.BATCH_SIZE,
            params_is_epochs=self.params.EPOCHS,
            params_is_batch_size=self.params.BATCH_SIZE,
            params_is_vocab=self.params.vocab,
            params_is_sent_length=self.params.sent_length,
            params_is_sen_length=self.params.sen_length
        )
        return eval_config

In [13]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [30]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def read_data(self):
        df = pd.read_csv(self.config.training_data)
        return df

    def target_independent_vars(self):
        df_inputData = self.read_data()
        X = df_inputData.drop('label',axis=1)
        y = df_inputData['label']
        return X,y

    def corpusData(self):

        global messages
        independent_vars,dependent_vars = self.target_independent_vars()

        messages = independent_vars.copy()

        messages.reset_index(inplace=True)

        nltk.download('stopwords')
        return messages

    def stemming(self):

        ps = PorterStemmer()

        preparedData = self.corpusData()
        corpus=[]
        
        for i in range(0,len(preparedData)):
            review = re.sub('[^a-zA-Z]', ' ',str(preparedData['title'][i]))
            review=review.lower()
            review=review.split()
            review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
            review=' '.join(review)
            corpus.append(review)

        return corpus

    def oneHot(self):
        stemmed = self.stemming()
        onehot_repr=[one_hot(words,self.config.params_is_vocab)for words in stemmed]
        return onehot_repr

    def padding_seq(self):
        onehotted = self.oneHot()
        embedded_docs = pad_sequences(onehotted,padding='pre',maxlen=self.config.params_is_sen_length)
        return embedded_docs

    def get_base_model(self):
        model = tf.keras.models.load_model(
            self.config.base_model_path
        )
        return model

    def splitData(self):

        ind_var,dep_var = self.target_independent_vars()
        emb_doc = self.padding_seq()

        X_final=np.array(emb_doc)
        y_final=np.array(dep_var)
        X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

        return X_train, X_test, y_train, y_test


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.padding_seq()
        trainX,testX,trainy,testy = self.splitData()
        self.score = model.evaluate(testX,testy)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [31]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2023-12-19 16:03:43,061: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-19 16:03:43,063: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-19 16:03:43,064: INFO: common: created directory at: artifacts]


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ayushyajnik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ayushyajnik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


: 